In [74]:
#!/usr/bin/python
# -*- encoding: utf-8 -*-


import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

import os.path as osp
import os
from PIL import Image
import numpy as np
import json

from transform import *



class CityScapes(Dataset):
    def __init__(self, rootpth, cropsize=(640, 480), mode='train', 
    randomscale=(0.125, 0.25, 0.375, 0.5, 0.675, 0.75, 0.875, 1.0, 1.25, 1.5), *args, **kwargs):
#         super(CityScapes, self).__init__(*args, **kwargs)
        assert mode in ('train', 'val', 'test', 'trainval')
        self.mode = mode
        print('self.mode', self.mode)
        self.ignore_lb = 255

        with open('./cityscapes_info.json', 'r') as fr:
            labels_info = json.load(fr)
        self.lb_map = {el['id']: el['trainId'] for el in labels_info}
        

        ## parse img directory
        self.imgs = {}
        imgnames = []
        impth = osp.join(rootpth, 'leftImg8bit', mode)
        folders = os.listdir(impth)
        for fd in folders:
            fdpth = osp.join(impth, fd)
            im_names = os.listdir(fdpth)
            names = [el.replace('_leftImg8bit.png', '') for el in im_names]
            impths = [osp.join(fdpth, el) for el in im_names]
            imgnames.extend(names)
            self.imgs.update(dict(zip(names, impths)))
        imgnames.remove(".ipynb_checkpoints")
        del self.imgs['.ipynb_checkpoints']
#         print(self.imgs)

        ## parse gt directory
        self.labels = {}
        gtnames = []
        gtpth = osp.join(rootpth, 'gtFine', mode)
        folders = os.listdir(gtpth)
        for fd in folders:
            fdpth = osp.join(gtpth, fd)
            lbnames = os.listdir(fdpth)
            lbnames = [el for el in lbnames if 'labelIds' in el]
            names = [el.replace('_gtFine_labelIds.png', '') for el in lbnames]
            lbpths = [osp.join(fdpth, el) for el in lbnames]
            gtnames.extend(names)
            self.labels.update(dict(zip(names, lbpths)))

        self.imnames = imgnames
        self.len = len(self.imnames)
        print('self.len', self.mode, self.len)
#         print(len(imgnames))
#         print("*********************************************")
#         print(len(gtnames))
#         for i in range(len(imgnames)):
#             if imgnames[i] not in gtnames:
#                 print("#####################################################################")
#                 print(imgnames[i],i)
#                 print("###################################################################")
#             else:
#                 print(imgnames[i],i)
        assert set(imgnames) == set(gtnames)
        assert set(self.imnames) == set(self.imgs.keys())
        assert set(self.imnames) == set(self.labels.keys())
        ## pre-processing
        self.to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        self.trans_train = Compose([
            ColorJitter(
                brightness = 0.5,
                contrast = 0.5,
                saturation = 0.5),
            HorizontalFlip(),
            # RandomScale((0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0)),
            RandomScale(randomscale),
            # RandomScale((0.125, 1)),
            # RandomScale((0.125, 0.25, 0.375, 0.5, 0.675, 0.75, 0.875, 1.0)),
            # RandomScale((0.125, 0.25, 0.375, 0.5, 0.675, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5)),
            RandomCrop(cropsize)
            ])


    def __getitem__(self, idx):
        fn  = self.imnames[idx]
        impth = self.imgs[fn]
        lbpth = self.labels[fn]
        img = Image.open(impth).convert('RGB')
        label = Image.open(lbpth)
        if self.mode == 'train' or self.mode == 'trainval':
            im_lb = dict(im = img, lb = label)
            im_lb = self.trans_train(im_lb)
            img, label = im_lb['im'], im_lb['lb']
        img = self.to_tensor(img)
        label = np.array(label).astype(np.int64)[np.newaxis, :]
        label = self.convert_labels(label)
        return img, label


    def __len__(self):
        return self.len


    def convert_labels(self, label):
        for k, v in self.lb_map.items():
            label[label == k] = v
        return label



if __name__ == "__main__":
    from tqdm import tqdm
    ds = CityScapes('/xiaoou/my_last_project/dataset/cityscapes', n_classes=19, mode='train')
    uni = []
    for im, lb in tqdm(ds):
        lb_uni = np.unique(lb).tolist()
        uni.extend(lb_uni)
    print(uni)
    print(set(uni))




  0%|          | 0/2975 [00:00<?, ?it/s]

self.mode train
self.len train 2975



100%|██████████| 2975/2975 [12:31<00:00,  3.96it/s]

[0, 1, 2, 5, 7, 8, 9, 11, 255, 0, 2, 5, 6, 7, 8, 9, 10, 11, 13, 255, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 255, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 255, 0, 5, 7, 8, 11, 13, 15, 255, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 18, 255, 0, 1, 2, 5, 8, 9, 10, 13, 15, 255, 0, 1, 2, 5, 7, 8, 9, 10, 11, 13, 18, 255, 0, 1, 2, 5, 6, 7, 8, 10, 11, 13, 255, 0, 1, 2, 5, 7, 8, 10, 11, 12, 13, 14, 17, 18, 255, 0, 1, 2, 5, 6, 7, 8, 9, 10, 13, 255, 0, 1, 2, 5, 6, 7, 8, 10, 11, 12, 13, 18, 255, 0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 255, 1, 2, 5, 6, 7, 8, 10, 11, 12, 13, 18, 255, 0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 18, 255, 0, 1, 2, 5, 7, 8, 11, 13, 18, 255, 0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 255, 0, 1, 2, 5, 7, 8, 9, 10, 11, 13, 255, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 255, 0, 1, 2, 5, 7, 8, 10, 11, 12, 14, 15, 17, 18, 255, 0, 2, 5, 6, 7, 8, 11, 12, 13, 14, 18, 255, 0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 13, 255, 0, 1, 2, 5, 6, 7, 8, 9, 11, 13, 17, 18, 255, 0, 1, 2, 

In [55]:
ls -a /xiaoou/my_last_project/dataset/cityscapes/leftImg8bit/train

./       bochum/   darmstadt/   hamburg/  krefeld/          stuttgart/  weimar/
../      bremen/   dusseldorf/  hanover/  monchengladbach/  tubingen/   zurich/
aachen/  cologne/  erfurt/      jena/     strasbourg/       ulm/


In [60]:
ls -a /xiaoou/my_last_project/dataset/cityscapes/leftImg8bit/train/aachen rm -rf ipynb_checkpoints/

ls: cannot access 'rm': No such file or directory
ls: cannot access 'ipynb_checkpoints/': No such file or directory
/xiaoou/my_last_project/dataset/cityscapes/leftImg8bit/train/aachen:
./                                    aachen_000145_000019_leftImg8bit.png
../                                   aachen_000097_000019_leftImg8bit.png
aachen_000028_000019_leftImg8bit.png  aachen_000107_000019_leftImg8bit.png
aachen_000131_000019_leftImg8bit.png  aachen_000013_000019_leftImg8bit.png
aachen_000149_000019_leftImg8bit.png  aachen_000078_000019_leftImg8bit.png
aachen_000144_000019_leftImg8bit.png  aachen_000150_000019_leftImg8bit.png
aachen_000067_000019_leftImg8bit.png  aachen_000099_000019_leftImg8bit.png
aachen_000110_000019_leftImg8bit.png  aachen_000022_000019_leftImg8bit.png
aachen_000006_000019_leftImg8bit.png  aachen_000142_000019_leftImg8bit.png
aachen_000014_000019_leftImg8bit.png  aachen_000151_000019_leftImg8bit.png
aachen_000116_000019_leftImg8bit.png  aachen_000058_000019_leftIm

In [63]:
ls -a /xiaoou/my_last_project/dataset/cityscapes/leftImg8bit/train/aachen/

./                                    aachen_000086_000019_leftImg8bit.png
../                                   aachen_000087_000019_leftImg8bit.png
.ipynb_checkpoints/                   aachen_000088_000019_leftImg8bit.png
aachen_000000_000019_leftImg8bit.png  aachen_000089_000019_leftImg8bit.png
aachen_000001_000019_leftImg8bit.png  aachen_000090_000019_leftImg8bit.png
aachen_000002_000019_leftImg8bit.png  aachen_000091_000019_leftImg8bit.png
aachen_000003_000019_leftImg8bit.png  aachen_000092_000019_leftImg8bit.png
aachen_000004_000019_leftImg8bit.png  aachen_000093_000019_leftImg8bit.png
aachen_000005_000019_leftImg8bit.png  aachen_000094_000019_leftImg8bit.png
aachen_000006_000019_leftImg8bit.png  aachen_000095_000019_leftImg8bit.png
aachen_000007_000019_leftImg8bit.png  aachen_000096_000019_leftImg8bit.png
aachen_000008_000019_leftImg8bit.png  aachen_000097_000019_leftImg8bit.png
aachen_000009_000019_leftImg8bit.png  aachen_000098_000019_leftImg8bit.png
aachen_000010_000019_left